In [5]:
import sys
sys.path.append("../../../helper_functions")
import web3py_utils as w3py
sys.path.pop()

import os
import pandas as pd
from dotenv import load_dotenv
load_dotenv()

endpoint = os.environ["OP_NODE"]

In [6]:
def collapse_signature(full_sig: str) -> str:
    """Simplify function signature to remove named params."""
    name_and_args = full_sig.strip().split("(")
    if len(name_and_args) != 2:
        return full_sig  # fallback in case of invalid format
    func_name = name_and_args[0]
    args_raw = name_and_args[1].rstrip(")")
    arg_types = [arg.strip().split()[0] for arg in args_raw.split(",") if arg]
    return f"{func_name}({','.join(arg_types)})"

def generate_snippet_data(sig_list, endpoint, opcode_filter=''):
    """Generate original, collapsed, and bytecode for each function signature."""
    results = []
    for sig in sig_list:
        collapsed = collapse_signature(sig)
        bytecode = w3py.get_bytecode_pattern(endpoint, collapsed, opcode_filter)
        cleaned_bytecode = bytecode.replace("'0x", "0x")
        results.append((sig, collapsed, cleaned_bytecode))
    return results


In [7]:
# Example signature set
transfer_signatures = [
    'transfer(address recipient, uint256 amount)',
    'transferFrom(address sender, address recipient, uint256 amount)',
    'sendERC20(address _token, address _to, uint256 _amount, uint256 _chainId)',
    'relayERC20(address _token, address _from, address _to, uint256 _amount)',
    'safeBatchTransferFrom(address from, address to, uint256[] ids, uint256[] amounts, bytes data)',
    'safeTransferFrom(address from, address to, uint256 id, uint256 amount, bytes data)'
]

In [8]:
data = generate_snippet_data(transfer_signatures, endpoint)

# Print SQL-style tuples
print("SQL-style tuples for query use:\n")
for original, collapsed, bytecode in data:
    print(f"('{collapsed}', {bytecode})")

# Print tabular for Google Sheets
print("\n\nTabular format for Google Sheets (Original Signature, Collapsed Signature, Bytecode Snippet):\n")
print("Original Signature\tCollapsed Signature\tBytecode Snippet")
for original, collapsed, bytecode in data:
    print(f"{original}\t{collapsed}\t{bytecode}")


SQL-style tuples for query use:

('transfer(address,uint256)', 0xa9059cbb)
('transferFrom(address,address,uint256)', 0x23b872dd)
('sendERC20(address,address,uint256,uint256)', 0xc1a433d8)
('relayERC20(address,address,address,uint256)', 0x7cfd6dbc)
('safeBatchTransferFrom(address,address,uint256[],uint256[],bytes)', 0x2eb2c2d6)
('safeTransferFrom(address,address,uint256,uint256,bytes)', 0xf242432a)


Tabular format for Google Sheets (Original Signature, Collapsed Signature, Bytecode Snippet):

Original Signature	Collapsed Signature	Bytecode Snippet
transfer(address recipient, uint256 amount)	transfer(address,uint256)	0xa9059cbb
transferFrom(address sender, address recipient, uint256 amount)	transferFrom(address,address,uint256)	0x23b872dd
sendERC20(address _token, address _to, uint256 _amount, uint256 _chainId)	sendERC20(address,address,uint256,uint256)	0xc1a433d8
relayERC20(address _token, address _from, address _to, uint256 _amount)	relayERC20(address,address,address,uint256)	0x7cfd6d